In [1]:
import os
import sys
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi

caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'ajna_docs', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from bhadrasana.models import engine

2022-05-23 09:46:34,354 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\access.log


In [2]:
SQL = 'SELECT * FROM ovr_itenstg'
df = pd.read_sql(SQL, engine)

In [3]:
df

,id,tg_id,numero,descricao,qtde,unidadedemedida,valor,ncm,marca_id,create_date,user_name,contramarca,modelo
0,2,9,1,123,1.0,1,1.00,11,None,2020-05-26 18:44:26,None,None,None
1,9,54,1,MÁSCARA FACIAL HIDRATANTE EM EMBALAGEM DE 25ML...,226000.0,0,2.60,33049910,None,2020-08-04 10:44:00,None,None,None
2,112,123,0,DIOCTIL FTALATO EM FLEXI-BAG D/C 20 TONELADAS ...,2.0,0,18000.00,29173200,None,2020-08-31 08:25:48,None,None,None
3,121,125,0,"SOLA DE CALÇADO, DE BORRACHA, N 39 A, REF XL B...",462.0,0,21.21,64062000,None,2020-09-11 14:32:05,None,None,None
4,123,125,2,"SOLA DE CALÇADO, DE BORRACHA, N 40 A, REF XL L...",98.0,0,21.21,64062000,None,2020-09-11 14:32:05,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15455,22826,590,365,BIJOUTERIAS DIVERSAS,4.0,1,50.00,71179000,None,2022-05-20 18:33:52,None,None,None
15456,22827,590,366,"AROMATIZANTES PARA VEÍCULOS, DIVERSAS MARCAS: ...",16.0,1,100.00,33079000,None,2022-05-20 18:33:52,None,None,None
15457,22828,590,367,"AROMATIZADOR DE AMBIENTE SPRAY, MARCA: WHITE BARN",1.0,1,100.00,33079000,None,2022-05-20 18:33:52,None,None,None
15458,22829,590,368,"REFIL DE AROMATIZADOR, DIVERSOS, MARCA: WALLFL...",12.0,1,50.00,33079000,None,2022-05-20 18:33:52,None,None,None


In [4]:
itenstg = df.dropna(subset=['ncm'])

In [5]:
len(itenstg)

15114

In [6]:
import nltk
import unicodedata

def remove_accents(input_str):  
    """ 
    Função converte string em bytes, mas antes normaliza string usando NFKD
    
    NFKD - decompõem em dois code point e analisa compatibilidades (sem ser canonicamente equivalente)
    https://docs.python.org/3/library/unicodedata.html
    """
    nfkd_form = unicodedata.normalize('NFKD', input_str.lower())
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii


def remove_pontuacao(doc):
    return ''.join([leter for leter in doc if leter not in ['.', ',', ':', ';', '-', '+']])

stop_words = nltk.corpus.stopwords.words('portuguese')

def tokenize(doc):
    doc_normalized = doc.lower()
    doc_normalized = remove_pontuacao(doc_normalized)
    doc_normalized = remove_accents(doc_normalized)
    return [word for word in doc_normalized.split() if (len(word) > 2 and word not in stop_words)]

In [7]:
tokenize('KIT DE CARTEIRA, COM RELÓGIO DE PULSO; Modelo: bolinha, Marca:bolão')

[b'kit',
 b'carteira',
 b'com',
 b'relogio',
 b'pulso',
 b'modelo',
 b'bolinha',
 b'marcabolao']

In [8]:
corpus = list(itenstg.descricao.values)
corpus_normalized = [tokenize(line) for line in corpus]
bm25n = BM25Okapi(corpus_normalized)
bm25n.get_top_n(tokenize('KIT DE CARTEIRA, COM RELÓGIO DE PULSO;'), corpus, n=10)

['KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DOS PERSONAGENS CARS, DA MARCA DISNEY – REF.:M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM FROZEN, DA MARCA DISNEY – REF.: M301A',
 'Kit de carteira com relógio imitação do personagem Miraculous . (Ref:M301A)',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DOS PERSONAGENS AVENGERS, DA MARCA MARVEL – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM MIRACULOUS, DA MARCA GLOBOSAT PROGRAMADORA – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DO PERSONAGEM BEN10, DA MARCA CARTOON NETWORK – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DO PERSONAGEM HOMEM ARANHA, DA MARCA MARVEL – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DAS PERSONAGENS PRINCESAS, DA MARCA DISNEY – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM

In [9]:
bm25n.get_top_n(tokenize('CONFECÇÕES USO ADULTO, LACOSTE, THOMMY HILFGER RALPH LAUREN E OUTRAS DIVERSAS'), corpus, n=10)

['CONFECÇÕES ADULTO LACOSTE, TOMMY HILFGER RALPH LAUREN E OUTRAS MARCAS',
 'CONFECÇÕES USO ADULTO, LACOSTE, THOMMY HILFGER, NIKE, ADIDAS, E OUTRAS MARCAS DIVERSAS',
 'Óculos adulto - calvin klein, thommy hilfger e outras marcas diversas',
 "CONFECÇÕES USO ADULTO, LEVI'S, TOMMY HILFGER, GAP, NIKE E OUTRAS MARCAS DIVERSAS",
 'CONFECÇÕES USO ADULTO, THOMMY HILFIGER, CALVIN KLEIN, POLO, ASSN E OUTRAS MARCAS DIVERSAS',
 'CONFECÇÕES USO ADULTO, NIKE, ADDIDAS E OUTRAS MARCAS DIVERSAS',
 'CONFECÇÕES USO ADULTO, polo, nike, adidas e outras marcas diversas',
 'CONFECÇÕES USO ADULTO',
 'CONFECÇÕES USO ADULTO, LEWIS, NBA E OUTRAS MARCAS',
 'CONFECÇÕES USO ADULTO, TOMMY, NIKE, SPEEDO, ADIDDAS E OUTRAS MARCAS DIVERSAS.']

In [10]:
tokenize('Perfume- 125ML - Hugo Boss')

[b'perfume', b'125ml', b'hugo', b'boss']

In [11]:
def monta_sugestoes(planilha):
    lista_dict = []
    planilha.rename(remove_accents)
    planilha = planilha.dropna(subset=['DESCRIÇÃO'])
    for row in planilha.iterrows():
        # print(row[1])
        row_ = {}
        row_['Item'] = row[1]['ITEM']
        row_['Código NCM'] = ''
        row_['Descrição'] = row[1]['DESCRIÇÃO']
        row_['Marca'] = ''
        row_['Modelo'] = ''
        row_['Unid. Medida'] = row[1]['Tipo de UNIDADE']
        row_['País Procedência'] = 249
        row_['País Origem'] = 994
        row_['Quantidade'] = row[1]['Quantidade']
        row_['Valor Unitário'] = ''
        lista_dict.append(row_)
        print(row_['Descrição'])
        doc_scores = bm25n.get_scores(tokenize(row_['Descrição']))
        indices = np.flip(np.argpartition(doc_scores, doc_scores.shape[0] - 3)[-5:])
        new_df = itenstg.iloc[np.flip(indices)]
        for new_row in new_df.iterrows():
            row_ = row_.copy()
            row_['Item'] = '-'
            row_['Descrição'] = new_row[1]['descricao']
            row_['Código NCM'] = new_row[1]['ncm']
            unidade = 'UN' if new_row[1]['unidadedemedida'] == 0 else 'KG'
            row_['Unid. Medida'] = unidade
            row_['Valor Unitário'] = new_row[1]['valor']
            lista_dict.append(row_)
    return pd.DataFrame(lista_dict)

In [12]:
def remove_accents2(input_str):  
    nfkd_form = unicodedata.normalize('NFKD', input_str.lower())
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

planilha.rename(remove_accents2, axis='columns')

NameError: name 'planilha' is not defined

In [13]:
diretorio = r'C:\Users\25052288840\Downloads\TGs'
for nome_planilha in os.listdir(diretorio):
    print(diretorio + '\\' + nome_planilha)
    planilha = pd.read_excel(diretorio + '\\' + nome_planilha)
    sugest_df = monta_sugestoes(planilha)
    sugest_df.to_excel(diretorio + '\\TG ' + nome_planilha, index=False)    

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'C:\\Users\\25052288840\\Downloads\\TGs'

In [14]:
bm25n.get_top_n(tokenize('Forno marca: Balmuda, modelo: K01M-KG'), corpus, n=10)

['FORNO ELETRICO, MARCA: MAINSTAYS, MODELO: 1010J-H15',
 'FORNO DE PAREDE, MARCA: BOSCH, MODELO: HBLP451UC',
 'FORNO DE PAREDE, MARCA: BOSCH, MODELO: HBL8453UC',
 'FORNO ELÉTRICO, MARCA: HAMILTON, MODELO: 22703H',
 'FORNO ELÉTRICO BLACK DECKER, MODELO: TO 1313SBD',
 'FORNO MICROONDAS, MARCA: BLACK DECKER, MODELO: EM720CB7',
 'FORNO ELETRICO, MARCA: ELITE GOURMET, MODELO: ETO-4510M',
 'FORNO ELETRICO 120V/(USADO)',
 'FORNO ELETRICO 120V/(USADO)',
 'FORNO - AF-250AD2  - KALORIK']

In [15]:
bm25n.get_top_n(tokenize('Lixadeira Marca: Dexter'), corpus, n=10)

['LIXADEIRA - GA4030 - MAKITA',
 'LIXADEIRA - R0520VS - BOSH',
 'LIXADEIRA - 64807 - WARRIOR',
 'LIXADEIRA - DCW210 - DE WALT',
 'LIXADEIRA ORBITAL - B05041 - MAKITA',
 'LIXADEIRA - RK7866 - SHOP SERIES',
 'lixadeira 7800 - porter cable',
 'LIXADEIRA, MARCA DEWALT, MODELO DWL6423',
 'LIXADEIRA DE UNHA - DR-203 - MELODY SUSIE',
 'LIXADEIRA ELÉTRICA - POWER SHOT PRO 0 9100K']

In [16]:
bm25n.get_top_n(tokenize('Microscópio Digital'), corpus, n=10)

['MICROSCÓPIO - FB20135 - STEM',
 'microscópio 52-piece - erly explorer series - IQ CREW',
 'Termômetro Digital',
 'conversor digital TV  - az',
 'TERMOMETRO DIGITAL - IHEALTH',
 'BALANÇA DIGITAL RENPHO',
 'Balança Digital - Leadzim',
 'LAREIRA DIGITAL - EF421SP',
 'BALANÇA DIGITAL - ESCALI',
 'TERMÔMETRO DIGITAL - DR. TALBOT´S']

In [17]:
bm25n.get_top_n(tokenize('Notebook  Asus'), corpus, n=10)

['NOTEBOOK - ASUS - USADO',
 'NOTEBOOK ASUS VIVOBOOK, MARCA ASUS, MODELO R543M',
 'tablet nexus 7 - C90KBC487109 - ASUS',
 'NOTEBOOK  (USADO)',
 'NOTEBOOK (USADO)',
 'notebook - HP - usado',
 'ROTEADOR, MARCA: ASUS, MODELO: RT-AC5300',
 'SUPORTE P/ NOTEBOOK, DE METAL/LOCTEK/DA1-TRAY',
 'NOTEBOOK DELL - AVARIADO',
 'NOTEBOOK - ADM A8 - LENOVO']

In [18]:
bm25n.get_top_n(tokenize('Lixadeira Dexter'), corpus, n=10)

['LIXADEIRA - GA4030 - MAKITA',
 'LIXADEIRA - R0520VS - BOSH',
 'LIXADEIRA - 64807 - WARRIOR',
 'LIXADEIRA - DCW210 - DE WALT',
 'LIXADEIRA ORBITAL - B05041 - MAKITA',
 'lixadeira 7800 - porter cable',
 'LIXADEIRA - RK7866 - SHOP SERIES',
 'LIXADEIRA DE UNHA - DR-203 - MELODY SUSIE',
 'LIXADEIRA, MARCA DEWALT, MODELO DWL6423',
 'LIXADEIRA ELÉTRICA - POWER SHOT PRO 0 9100K']

In [19]:
bm25n.get_top_n(tokenize('Espremedor black modelo: CJ625'), corpus, n=10)

['ESPREMEDOR DE FRUTAS, MARCA: CUISINART, MODELO: CCJ500',
 'ESPREMEDOR - K37-010 - HOMELEADER',
 'ESPREMEDOR DE FRUTAS 120V/(USADO)',
 'ESPREMEDOR DE FRUTAS 120/(USADO)',
 'ESPREMEDOR MANUAL - CIRILLO',
 'Espremedor - CS1000 - Citri Star',
 'ESPREMEDOR DE FRUTAS - PKJCR305 - NUTRICHEF',
 'PERFUME 100ML - BLACK IS BLACK - NUPARFUMS',
 'APARELHO ESPREMEDOR DE FRUTA ELÉTRICO - ELITE',
 'ESPREMEDOR DE FRUTAS - 67601-A - HAMILTON BEACH']

In [20]:
SQL = 'SELECT title as ncm, contents as descricao FROM laudo_ncm'
df_ncm = pd.read_sql(SQL, engine)
itensncm = df_ncm.dropna(subset=['ncm'])

In [21]:
print(len(df_ncm), len(itensncm))

10147 10147


In [22]:
corpus_ncm = list(itensncm.descricao.values)
corpus_ncm_normalized = [tokenize(line) for line in corpus_ncm]
bm25n_ncm = BM25Okapi(corpus_ncm_normalized)

In [23]:
bm25n_ncm.get_top_n(tokenize('KIT DE CARTEIRA COM RELÓGIO DE PULSO'), corpus_ncm, n=10)

['--\tNão combinados com um aparelho de gravação ou de reprodução de som, mas combinados com um relógio -\tOutros: Aparelhos receptores para radiodifusão, mesmo combinados num mesmo invólucro, com um aparelho de gravação ou de reprodução de som, ou com um relógio.',
 'Combinado com relógio --\tOutros -\tAparelhos receptores de radiodifusão suscetíveis de funcionarem sem fonte externa de energia: Aparelhos receptores para radiodifusão, mesmo combinados num mesmo invólucro, com um aparelho de gravação ou de reprodução de som, ou com um relógio.',
 '--\tOutros -\tOutros relógios de pulso, mesmo com contador de tempo incorporado: Relógios de pulso, relógios de bolso e relógios semelhantes (incluindo os contadores de tempo dos mesmos tipos), exceto os da posição 91.01.',
 '--\tDe corda automática -\tOutros relógios de pulso, mesmo com contador de tempo incorporado: Relógios de pulso, relógios de bolso e relógios semelhantes (incluindo os contadores de tempo dos mesmos tipos), exceto os da p

# Testar com lemmatização

In [54]:
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()
stop_words = nltk.corpus.stopwords.words('portuguese')

def tokenize_stem(doc):
    doc_normalized = doc.lower()
    doc_normalized = remove_pontuacao(doc_normalized)
    doc_normalized = remove_accents(doc_normalized)
    return [stemmer.stem(word.decode()) for word in doc_normalized.split() if (len(word) > 2 and word not in stop_words)]
tokenize_stem('Geladeira, serie HR42617091, fabricação México')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\25052288840\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


['gelad', 'seri', 'hr42617091', 'fabricaca', 'mexic']

In [55]:
corpus_stem = [tokenize_stem(line) for line in corpus]
bm25_stem = BM25Okapi(corpus_stem)

In [56]:
bm25_stem.get_top_n(tokenize_stem('KIT DE CARTEIRA, COM RELÓGIO DE PULSO;'), corpus, n=10)

['KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM FROZEN, DA MARCA DISNEY – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DOS PERSONAGENS CARS, DA MARCA DISNEY – REF.:M301A',
 'Kit de carteira com relógio imitação do personagem Miraculous . (Ref:M301A)',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM MIRACULOUS, DA MARCA GLOBOSAT PROGRAMADORA – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM PRINCESA SOFIA, DA MARCA DISNEY – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DAS PERSONAGENS PRINCESAS, DA MARCA DISNEY – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DO PERSONAGEM BEN10, DA MARCA CARTOON NETWORK – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DO PERSONAGEM HOMEM ARANHA, DA MARCA MARVEL – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DOS PERSO

In [57]:
bm25_stem.get_top_n(tokenize_stem('geladeira'), corpus, n=10)

['GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY',
 'IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( USADO )',
 'Mini geladeira automotiva - K6101WH - Personal Chiller',
 'GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA SIBIR/USADO',
 'MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MODELO: EFMIS129',
 'Adesivo terapêutico Fantu',
 'Óculos de sol IMITAÇÃO DA MARCA RAY BAN',
 'Óculos de sol IMITAÇÃO DA MARCA CHANEL',
 'Selo IMITAÇÃO DA ANATEL - número 03864-18-11469',
 'TV Box MXQ Pro 4K - ADULTERADO - PRODUTO PROIBIDO']

In [58]:
tokenize_stem('Geladeira, serie HR42617091, fabricação México')

['gelad', 'seri', 'hr42617091', 'fabricaca', 'mexic']

In [59]:
tokenize('Geladeira, serie HR42617091, fabricação México')

[b'geladeira', b'serie', b'hr42617091', b'fabricacao', b'mexico']

In [48]:
bm25n.get_top_n(tokenize('geladeira mexico'), corpus, n=10)

['GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY',
 'AMORTECEDOR AUTOMOTIVO MODELO: 018166 ORIGEM: MÉXICO',
 'TEQUILA PATRÓN 100% DE AGAVE 750ML, ORIGEM MEXICO',
 'GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA SIBIR/USADO',
 'IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( USADO )',
 'Mini geladeira automotiva - K6101WH - Personal Chiller',
 'MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MODELO: EFMIS129',
 'Selo IMITAÇÃO DA ANATEL - número 03864-18-11469',
 'TV Box MXQ Pro 4K - ADULTERADO - PRODUTO PROIBIDO',
 'Adesivo terapêutico Fantu']

In [60]:
bm25_stem.get_top_n(tokenize_stem('geladeira mexico'), corpus, n=10)

['GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY',
 'AMORTECEDOR AUTOMOTIVO MODELO: 018166 ORIGEM: MÉXICO',
 'TEQUILA PATRÓN 100% DE AGAVE 750ML, ORIGEM MEXICO',
 'GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA SIBIR/USADO',
 'IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( USADO )',
 'Mini geladeira automotiva - K6101WH - Personal Chiller',
 'MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MODELO: EFMIS129',
 'Selo IMITAÇÃO DA ANATEL - número 03864-18-11469',
 'TV Box MXQ Pro 4K - ADULTERADO - PRODUTO PROIBIDO',
 'Adesivo terapêutico Fantu']

In [61]:
bm25_stem.get_top_n(tokenize_stem('bolsas'), corpus, n=10)

['BOLSA FEMININA (JOGO C/ 02 BOLSAS E 01 MOEDEIRO)/LC063/LC065/LC066/LC068',
 'BOLSA FEMININA/DIVERSOS',
 'BOLSA - SPLENDID',
 'BOLSA - ALDO',
 'BOLSA - COACH',
 'BOLSA - LEVIS',
 'BOLSA TÉRMICA/DIVERSOS',
 'BOLSAS USADA',
 'BOLSA - GUESS',
 'BOLSA - NAUTICA']

In [62]:
bm25_stem.get_top_n(tokenize_stem('bolsas femininas'), corpus, n=10)

['BOLSA FEMININA (JOGO C/ 02 BOLSAS E 01 MOEDEIRO)/LC063/LC065/LC066/LC068',
 'BOLSA FEMININA, EM NYLON/SPORT/MOOTSIES/N-1/N-2/N-3/N-4',
 'BOLSA FEMININA, EM TECIDO/KD10',
 'BOLSA FEMININA, DOBRÁVEL/DIVERSOS',
 'BOLSA FEMININA, IMITAÇÃO DE COURO (JOGO C/ 02 BOLSAS E 01 MOEDEIRO)/BARBARA/BV063/BV065/BV066/BV068',
 'BOLSA FEMININA, DE MATERIAL SINTÉTICO/STYLE-773',
 'BOLSA FEMININA, IMITAÇÃO DE COURO/LC179/LC181/LC185/LC187',
 'BOLSA FEMININA, DIVERSAS CORES/760',
 'BOLSA FEMININA, DIVERSAS CORES/751',
 'BOLSA FEMININA, DIVERSAS CORES/772/773']

In [63]:
bm25n.get_top_n(tokenize('bolsas femininas'), corpus, n=10)

['BOLSAS USADA',
 'BOLSAS DIVERSAS',
 "CARTEIRAS FEMININAS - VICTORIA SECRET'S, KIPLING E OUTRAS MARCAS",
 'BOLSAS, MARCA: BEBE',
 'BOLSAS DIVERSAS MARCAS',
 'BOLSAS, MARCA: CORELLO',
 'BOLSAS, MARCA: GUESS',
 'BOLSAS, MARCA: FOSSIL',
 'BOLSAS, MARCA: CHANEL',
 '07 CXS BOLSAS E MOCHILAS/(USADO)']

In [69]:
for linha in corpus:
    if 'geladeira' in linha.lower():
        print(linha)

GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA SIBIR/USADO
Mini geladeira automotiva - K6101WH - Personal Chiller
IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( USADO )
GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY
MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MODELO: EFMIS129


In [70]:
bm25n.get_top_n(tokenize('geladeira'), corpus, n=10)

['GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY',
 'IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( USADO )',
 'Mini geladeira automotiva - K6101WH - Personal Chiller',
 'GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA SIBIR/USADO',
 'MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MODELO: EFMIS129',
 'Adesivo terapêutico Fantu',
 'Óculos de sol IMITAÇÃO DA MARCA RAY BAN',
 'Óculos de sol IMITAÇÃO DA MARCA CHANEL',
 'Selo IMITAÇÃO DA ANATEL - número 03864-18-11469',
 'TV Box MXQ Pro 4K - ADULTERADO - PRODUTO PROIBIDO']

In [71]:
bm25_stem.get_top_n(tokenize_stem('geladeira'), corpus, n=10)

['GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY',
 'IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( USADO )',
 'Mini geladeira automotiva - K6101WH - Personal Chiller',
 'GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA SIBIR/USADO',
 'MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MODELO: EFMIS129',
 'Adesivo terapêutico Fantu',
 'Óculos de sol IMITAÇÃO DA MARCA RAY BAN',
 'Óculos de sol IMITAÇÃO DA MARCA CHANEL',
 'Selo IMITAÇÃO DA ANATEL - número 03864-18-11469',
 'TV Box MXQ Pro 4K - ADULTERADO - PRODUTO PROIBIDO']

In [72]:
tokenize('Mini geladeira automotiva - K6101WH - Personal Chiller')

[b'mini', b'geladeira', b'automotiva', b'k6101wh', b'personal', b'chiller']

In [73]:
tokenize_stem('Mini geladeira automotiva - K6101WH - Personal Chiller')

['min', 'gelad', 'automo', 'k6101wh', 'person', 'chill']

In [83]:
def monta_sugestoes(texto, n_rows=10):
    doc_scores = bm25n.get_scores(tokenize(texto))
    indices = np.flip(np.argpartition(doc_scores, doc_scores.shape[0] - 2)[-n_rows:])
    indices_menoszero = []
    for indice in indices:
        if doc_scores[indice] == 0.:
            break
        indices_menoszero.append(indice)
    new_df = itenstg.iloc[indices_menoszero]
    # print(new_df.columns)
    # new_df['get_unidade'] = new_df.unidadedemedida.apply(Enumerado.unidadeMedida)
    return new_df

In [84]:
monta_sugestoes('geladeira')

,id,tg_id,numero,descricao,qtde,unidadedemedida,valor,ncm,marca_id,create_date,user_name,contramarca,modelo
10077,13858,542,1,GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY,1.0,0,1800.00,84182900,None,2022-01-25 14:23:04,None,None,None
7292,7752,455,23,GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA...,1.0,0,1543.92,84182100,None,2021-08-10 11:26:00,None,None,None
8659,9656,505,116,Mini geladeira automotiva - K6101WH - Personal...,1.0,0,800.00,84182900,None,2021-09-30 14:55:28,None,None,None
9639,13395,557,1,"IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( ...",30.0,0,0.06,39264000,None,2021-12-17 14:43:27,None,None,None
10728,15457,504,3,"MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MO...",2.0,0,250.00,84181000,None,2022-02-15 15:07:52,None,None,None


In [122]:
def monta_sugestoes_stem(texto, n_rows=10):
    doc_scores = bm25_stem.get_scores(tokenize_stem(texto))
    new_df = itenstg.copy()
    new_df['pontuação'] = doc_scores
    new_df = new_df[new_df['pontuação'] > 0.]
    new_df.sort_values(axis=0, inplace=True, ascending=False, by='pontuação')
    return new_df

In [123]:
monta_sugestoes_stem('geladeira')

,id,tg_id,numero,descricao,qtde,unidadedemedida,valor,ncm,marca_id,create_date,user_name,contramarca,modelo,pontuação
10077,13858,542,1,GELADEIRA PORTÁTIL - EP24755US-BK - COSTWAY,1.0,0,1800.00,84182900,None,2022-01-25 14:23:04,None,None,None,9.713738
7292,7752,455,23,GELADEIRA NR EK1222-E 1504000120 (230 V)/MARCA...,1.0,0,1543.92,84182100,None,2021-08-10 11:26:00,None,None,None,8.354852
8659,9656,505,116,Mini geladeira automotiva - K6101WH - Personal...,1.0,0,800.00,84182900,None,2021-09-30 14:55:28,None,None,None,8.354852
9639,13395,557,1,"IMÃ DE GELADEIRA DIVERSOS MODELOS, TAMANHOS ( ...",30.0,0,0.06,39264000,None,2021-12-17 14:43:27,None,None,None,8.354852
10728,15457,504,3,"MINI GELADEIRA PORTÁTIL, MARCA: FRIGIDAIRE, MO...",2.0,0,250.00,84181000,None,2022-02-15 15:07:52,None,None,None,7.808663


In [85]:
doc_scores = bm25_stem.get_scores(tokenize_stem('geladeira'))

In [94]:
np.where(doc_scores > 0)

(array([ 6992,  8359,  9299,  9737, 10388], dtype=int64),)

In [95]:
doc_scores[np.where(doc_scores > 0)]

array([8.35485228, 8.35485228, 8.35485228, 9.71373829, 7.80866283])

In [101]:
np.argpartition(doc_scores,(1, doc_scores.shape[0] - 1))[-10:]

array([15104, 15105, 15106, 15107, 15108, 15109, 15110, 15111, 15112,
        9737], dtype=int64)

In [103]:
max_scores = doc_scores[np.where(doc_scores > 0)]
max_scores

array([8.35485228, 8.35485228, 8.35485228, 9.71373829, 7.80866283])

In [110]:
np.argpartition(max_scores, 3)

array([4, 0, 1, 2, 3], dtype=int64)

In [111]:
np.argmax(max_scores)

3